In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd

from battleship.grammar import BattleshipGrammar
from battleship.scoring import compute_score, compute_score_parallel
from battleship.board import Board

import matplotlib.pyplot as plt

In [3]:
grammar = BattleshipGrammar(include_lambdas=False)

Sampling

In [6]:
import time 
from math import ceil

dataDict = {
    'program':[],
    'board_id':[],
    'score':[],
    'max_depth':[],
    'sample_length':[]
}
df = pd.DataFrame(dataDict)

if not os.path.isfile('enumData.csv'):
   df.to_csv('enumData.csv', header='column_names')


In [7]:

def sample_baseline(cores=int(os.cpu_count()/2),sample_length=5,depth=8):
    scores = []
    for id in range(1,18+1):
        start_time = time.time()
        print(f"board {id}", end=" | ") 
        acceptable_programs = []
        while len(acceptable_programs) != sample_length: #Rejection sampling of {sample_length} programs of depth {max_depth}
            try:
                program = grammar.sample(depth)
            except:
                continue
            if program != None: #Sometimes the grammar returns a NoneType (bug?) so this is necessary 
                if "(" in program or ")" in program: #Ignores all single-token questions: a single-token question won't have brackets, so if we know a response has brackets, it is not single-token. 
                #A more robust "min-depth" parameter would parse the generated program, but this will do for one-token answers 
                    acceptable_programs.append(program)
        print(f"finished generating programs in {round(time.time()-start_time,2)}s", end=" | ")
     
        if cores != 1:
            #If more than one core is used, breaks it up into chunks of 50 programs to score in parallel as to not overwork the machine and kill the EC2 instance or the kernel.
            for i in range(ceil(sample_length/50)):
                program_scores = compute_score_parallel(programs=acceptable_programs[i*50:((i+1)*50)],board=Board.from_trial_id(id),processes=cores,show_progress=False)
                scores.extend(program_scores)
                dataDict['program'].extend(acceptable_programs[i*50:((i+1)*50)])
                dataDict['board_id'].extend([id for i in range(len(program_scores))])
                dataDict['score'].extend(program_scores)
                dataDict['max_depth'].extend([depth for i in range(len(program_scores))])
                dataDict['sample_length'].extend([sample_length for i in range(len(program_scores))])
        else:
            #If only one core is used, computes programs scores sequentially
            for prog in acceptable_programs:
                score = compute_score(program=prog, board=Board.from_trial_id(id))
                scores.append(score)
        print(f"finished scoring in {round(time.time()-start_time,2)}s from the start")

    df = pd.DataFrame(dataDict)
    df.to_csv('sampleData.csv', mode='a', header=False)
    print(f"finished {sample_length}-shot sampling at depth {depth}")
    return df

In [ ]:
samples = [1,5,10,20,25,50,100,250,500,1000,2000]
depths = [5,6,7,8,9,10]

for depth in depths:
    for sample_num in samples:
        sample_baseline(sample_length=sample_num,depth=depth)

In [ ]:
df = pd.read_csv("enumData.csv")